In [13]:
import pandas as pd
import shopify
import requests
import json
import binascii
import os

In [14]:
dir(shopify)

['AccessScope',
 'Address',
 'ApiAccess',
 'ApiAccessError',
 'ApiPermission',
 'ApiVersion',
 'ApplicationCharge',
 'ApplicationCredit',
 'Article',
 'Asset',
 'Balance',
 'BillingAddress',
 'Blog',
 'CarrierService',
 'Cart',
 'Checkout',
 'Collect',
 'CollectionListing',
 'CollectionPublication',
 'Comment',
 'Country',
 'Currency',
 'CustomCollection',
 'Customer',
 'CustomerGroup',
 'CustomerInvite',
 'CustomerSavedSearch',
 'DiscountCode',
 'DiscountCodeCreation',
 'Disputes',
 'DraftOrder',
 'DraftOrderInvoice',
 'Event',
 'Fulfillment',
 'FulfillmentEvent',
 'FulfillmentOrders',
 'FulfillmentService',
 'GiftCard',
 'GiftCardAdjustment',
 'GraphQL',
 'Image',
 'InvalidVersionError',
 'InventoryItem',
 'InventoryLevel',
 'Limits',
 'LineItem',
 'Location',
 'MarketingEvent',
 'Metafield',
 'NoteAttribute',
 'Option',
 'Order',
 'OrderRisk',
 'Page',
 'PaginatedIterator',
 'PaymentDetails',
 'Payouts',
 'Policy',
 'PriceRule',
 'Product',
 'ProductListing',
 'ProductPublication',


In [15]:
url_prd = "https://slgjlkglkjglkjkljlkfgjlzkjlk0"
data_prd = requests.get(url_prd).json()

In [16]:
product_list = []
for item in data_prd["products"]:
    id = item["id"]
    vendor = item["vendor"]
    type = item["product_type"]
    title = item["title"]
    status = item["status"]
    created = item["created_at"]
    published = item["published_at"]
    updated = item["updated_at"]
    for variant in item["variants"]:
        price = variant["compare_at_price"]
        disc_price = variant["price"]
        sku = variant["sku"]
        barcode = variant["barcode"]
        invent_id = variant["inventory_item_id"]
        set_quant = ["old_inventory_quantity"]
        avl_quant = ["inventory_quantity"]
        shipping = ["requires_shipping"] 

        product = {"id" : id,
                "vendor" : vendor,
                "type" : type,
                "title" : title,
                "price" : price,
                "disc_price" : disc_price,
                "set_quant" : set_quant,
                "avl_quant" : avl_quant,
                "status" : status,
                "sku" : sku,
                "barcode" : barcode,
                "invent_id" : invent_id,
                "shipping" : shipping,
                "created" : created,
                "published" : published,
                "updated" : updated}
        product_list.append(product)
df = pd.DataFrame(product_list)
df.to_csv("shopify.csv", index=False)
print("saved to file.")     

saved to file.


In [17]:
df = pd.read_csv("shopify.csv")
df.shape

(250, 16)

In [18]:
df.sku.value_counts()

7050838                                   2
7050839                                   2
air-armada-102-L-S-C                      2
7050808                                   2
7050835                                   2
                                         ..
8260 power sectional (home elegant)       1
85703                                     1
868-3237-LIGHT GRAY TABLE AND 4 CHAIRS    1
881183 oficce chair                       1
B4610-11                                  1
Name: sku, Length: 239, dtype: int64

In [19]:
df.head(1)

,id,vendor,type,title,price,disc_price,set_quant,avl_quant,status,sku,barcode,invent_id,shipping,created,published,updated
0,6893888503985,Ashley,Mattress,10 Inch Chime Elite Full Mattress White/Blue,779.0,389.0,['old_inventory_quantity'],['inventory_quantity'],active,M67321,2.405250e+10,42658759540913,['requires_shipping'],2021-08-19T19:01:25-04:00,2021-08-20T11:25:04-04:00,2023-01-14T19:06:04-05:00


In [20]:
def get_all_resources(resource, **kwargs):
    resource_count = resource.count(**kwargs)
    resources = []
    if resource_count > 0:
        for page in range(1, ((resource_count-1) // 250) + 2):
            kwargs.update({"limit" : 250, "page" : page})
            resources.extend(resource.find(**kwargs))
    return resources

In [21]:
products = get_all_resources(shopify.Product)

TypeError: expected string or bytes-like object